In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cancerfinetuning/val.csv
/kaggle/input/cancerfinetuning/train.csv
/kaggle/input/cancerfinetuning/test.csv


In [15]:
import os
for dirname, _, filenames in os.walk('/kaggle/output'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/output/baseline_output/checkpoint-114/trainer_state.json
/kaggle/output/baseline_output/checkpoint-114/vocab.txt
/kaggle/output/baseline_output/checkpoint-114/special_tokens_map.json
/kaggle/output/baseline_output/checkpoint-114/optimizer.pt
/kaggle/output/baseline_output/checkpoint-114/rng_state.pth
/kaggle/output/baseline_output/checkpoint-114/training_args.bin
/kaggle/output/baseline_output/checkpoint-114/config.json
/kaggle/output/baseline_output/checkpoint-114/tokenizer.json
/kaggle/output/baseline_output/checkpoint-114/tokenizer_config.json
/kaggle/output/baseline_output/checkpoint-114/scheduler.pt
/kaggle/output/baseline_output/checkpoint-114/model.safetensors
/kaggle/output/baseline_output/checkpoint-38/trainer_state.json
/kaggle/output/baseline_output/checkpoint-38/vocab.txt
/kaggle/output/baseline_output/checkpoint-38/special_tokens_map.json
/kaggle/output/baseline_output/checkpoint-38/optimizer.pt
/kaggle/output/baseline_output/checkpoint-38/rng_state.pth
/kaggle/out

In [16]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer, TrainingArguments
)
from datasets import Dataset
# import evaluate
import json

In [17]:
# Load CSVs
train_df = pd.read_csv("/kaggle/input/cancerfinetuning/train.csv")
val_df = pd.read_csv("/kaggle/input/cancerfinetuning/val.csv")
test_df = pd.read_csv("/kaggle/input/cancerfinetuning/test.csv")

In [18]:
config = {
  "model_name": "distilbert-base-uncased",
  "max_length": 512,
  "train_batch_size": 8,
  "eval_batch_size": 8,
  "learning_rate": 2e-5,
  "num_train_epochs": 3,
  "output_dir": "/kaggle/working/baseline_output"
}


In [19]:
# Tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained(config["model_name"])

def tokenize(batch):
    return tokenizer(batch['abstract'], padding="max_length", truncation=True, max_length=config["max_length"])

train_ds = Dataset.from_pandas(train_df[['abstract', 'label']])
val_ds = Dataset.from_pandas(val_df[['abstract', 'label']])
test_ds = Dataset.from_pandas(test_df[['abstract', 'label']])

train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [20]:

# Model
model = DistilBertForSequenceClassification.from_pretrained(config["model_name"], num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.51.1",
  "vocab_size": 30522
}

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/model.safetensors
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification

In [21]:
from transformers import logging
logging.set_verbosity_info()  # <-- Enable detailed logs

training_args = TrainingArguments(
    output_dir=config["output_dir"],
    per_device_train_batch_size=config["train_batch_size"],
    per_device_eval_batch_size=config["eval_batch_size"],
    learning_rate=config["learning_rate"],
    num_train_epochs=config["num_train_epochs"],
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,  # Log every 10 steps
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"  # Disable wandb or other integrations
)


PyTorch: setting up devices


In [22]:
# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = np.mean(preds == labels)
    return {"accuracy": acc}

In [23]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train
trainer.train()

/tmp/ipykernel_31/3239434369.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: abstract. If abstract are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 600
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Training with DataParallel so batch size has been adjusted to: 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 114
  Number of trainable parameters = 66,955,010
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will ins

Epoch,Training Loss,Validation Loss,Accuracy
1,0.629000,0.386884,0.940000
2,0.178200,0.165375,0.945000
3,0.114800,0.144650,0.955000


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: abstract. If abstract are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 200
  Batch size = 16
Saving model checkpoint to /kaggle/working/baseline_output/checkpoint-38
Configuration saved in /kaggle/working/baseline_output/checkpoint-38/config.json
Model weights saved in /kaggle/working/baseline_output/checkpoint-38/model.safetensors
tokenizer config file saved in /kaggle/working/baseline_output/checkpoint-38/tokenizer_config.json
Special tokens file saved in /kaggle/working/baseline_output/checkpoint-38/special_tokens_map.json
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  wa

TrainOutput(global_step=114, training_loss=0.34004520794801546, metrics={'train_runtime': 65.8703, 'train_samples_per_second': 27.326, 'train_steps_per_second': 1.731, 'total_flos': 238441317580800.0, 'train_loss': 0.34004520794801546, 'epoch': 3.0})

In [34]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

# Summary metrics and confusion matrices
summary_records = []
conf_matrices = {}

def evaluate_model(split_name, dataset, df, model_tag):
    preds = trainer.predict(dataset)
    y_true = preds.label_ids
    y_pred = torch.argmax(torch.tensor(preds.predictions), dim=1).numpy()

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)

    summary_records.append({
        "Model": model_tag,
        "Split": split_name,
        "Accuracy": f"{acc*100:.2f}%",
        "F1-score": f"{f1:.2f}"
    })

    cm_df = pd.DataFrame(
        cm,
        index=["Actual Cancer", "Actual Non-Cancer"],
        columns=["Predicted Cancer", "Predicted Non-Cancer"]
    )
    conf_matrices[f"{split_name}_{model_tag}"] = cm_df

# Example: for LoRA model (repeat for baseline)
evaluate_model("train", train_ds, train_df, "distilbert")
evaluate_model("val", val_ds, val_df, "distilbert")
evaluate_model("test", test_ds, test_df, "distilbert")

# Save everything to a single Excel
output_path = "model_performance_report.xlsx"
summary_df = pd.DataFrame(summary_records)

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    summary_df.to_excel(writer, sheet_name="Metrics", index=False)

    # Create Confusion_Matrix sheet
    workbook = writer.book
    worksheet = workbook.create_sheet("Confusion_Matrix")

    for name, cm_df in conf_matrices.items():
        # Add matrix title
        worksheet.append([name])  # writes in first column
        for row in dataframe_to_rows(cm_df, index=True, header=True):
            worksheet.append(row)
        worksheet.append([])  # Add empty row between blocks

    # Remove default empty sheet if exists
    if "Sheet" in workbook.sheetnames:
        del workbook["Sheet"]

print(f"✅ All metrics saved to {output_path}")


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: abstract. If abstract are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Prediction *****
  Num examples = 600
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: abstract. If abstract are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Prediction *****
  Num examples = 200
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: abstract. If abstract are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Prediction *****
  Num examples = 200
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


✅ All metrics saved to model_performance_report.xlsx


In [29]:
import torch.nn.functional as F
import numpy as np
import json

# Helper function to generate prediction results
def get_predictions(dataset, df, label_map_rev):
    preds = trainer.predict(dataset)
    logits = preds.predictions
    probs = F.softmax(torch.tensor(logits), dim=1).numpy()
    label_ids = preds.label_ids

    results = []
    for i, (prob, true_label) in enumerate(zip(probs, label_ids)):
        pred_label_idx = np.argmax(prob)
        predicted_labels = [label_map_rev[pred_label_idx]]
        confidence_scores = {
            label_map_rev[0]: round(float(prob[0]), 2),
            label_map_rev[1]: round(float(prob[1]), 2),
        }
        results.append({
            "id": str(df.iloc[i]['id']),
            "split": split_name,  # Tag with train/val/test
            "predicted_labels": predicted_labels,
            "confidence_scores": confidence_scores
        })
    return results

# Label map
label_map_rev = {0: "Non-Cancer", 1: "Cancer"}

# Add split name before calling
split_name = "train"
train_results = get_predictions(train_ds, train_df.reset_index(drop=True), label_map_rev)

split_name = "val"
val_results = get_predictions(val_ds, val_df.reset_index(drop=True), label_map_rev)

split_name = "test"
test_results = get_predictions(test_ds, test_df.reset_index(drop=True), label_map_rev)

# Combine and save
combined_results = train_results + val_results + test_results

with open("/kaggle/working/all_predictions.json", "w") as f:
    json.dump(combined_results, f, indent=2)

print("✅ Saved all_predictions.json with predictions for train, val, and test.")


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: abstract. If abstract are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Prediction *****
  Num examples = 600
  Batch size = 16
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: abstract. If abstract are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Prediction *****
  Num examples = 200
  Batch size = 16


The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: abstract. If abstract are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Prediction *****
  Num examples = 200
  Batch size = 16


✅ Saved all_predictions.json with predictions for train, val, and test.


In [30]:
model.save_pretrained("/kaggle/working/baseline_output/")
tokenizer.save_pretrained("/kaggle/working/baseline_output/")


Configuration saved in /kaggle/working/baseline_output/config.json
Model weights saved in /kaggle/working/baseline_output/model.safetensors
tokenizer config file saved in /kaggle/working/baseline_output/tokenizer_config.json
Special tokens file saved in /kaggle/working/baseline_output/special_tokens_map.json


('/kaggle/working/baseline_output/tokenizer_config.json',
 '/kaggle/working/baseline_output/special_tokens_map.json',
 '/kaggle/working/baseline_output/vocab.txt',
 '/kaggle/working/baseline_output/added_tokens.json',
 '/kaggle/working/baseline_output/tokenizer.json')

In [31]:
import pickle

with open("/kaggle/working/distilbert_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("/kaggle/working/distilbert_tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
